In [1]:
import hydra
import torch
import torchaudio

with hydra.initialize(config_path="..", version_base=None):
    cond_cfg = hydra.compose(config_name="config", overrides=['exp=train_musdb_conditional'])
model = hydra.utils.instantiate(cond_cfg["model"])

ckpt = torch.load("../ckpts/musdb-conditional_epoch=423.ckpt", map_location="cpu")
model.load_state_dict(ckpt['state_dict'], strict=False)
model = model.cuda()

/home/emilian/miniconda3/envs/componet/lib/python3.11/site-packages/diffusers/models/transformer_2d.py:20: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 0.29. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.transformers.transformer_2d import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "0.29", deprecation_message)
/home/emilian/miniconda3/envs/componet/lib/python3.11/site-packages/diffusers/models/transformer_2d.py:25: FutureWarning: `Transformer2DModel` is deprecated and will be removed in version 0.29. Importing `Transformer2DModel` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.transformers.transformer_2d import Transformer2DModel`, instead.
  deprecate("Transformer2DModel", "0.29", deprecation_message)
/hom

In [5]:
y, sr = torchaudio.load("in.wav") # load you audio input
assert sr == 16000

y = torch.clip(y, -1, 1)
y_melspec = model.stft.cpu().mel_spectrogram(y.cpu())[0]
y_latents = model.vae.encode(y_melspec.cuda().permute(0, 2, 1).unsqueeze(1)).latent_dist.sample()
y_latents = y_latents * model.vae.config.scaling_factor

samples = model.pipeline(["in: other_1, vocals_1; out: vocals_1"], num_inference_steps=150,
         guidance_scale=1.0, audio_length_in_s=10.23, controlnet_cond=y_latents.cuda()).audios
torchaudio.save(f"out.wav", torch.tensor(samples[0]).unsqueeze(0), sample_rate=sr)

  0%|          | 0/150 [00:00<?, ?it/s]